# Tugas Pemrograman Kecil 1
Mata Kuliah: Temu-Balik Informasi

Author: Luthfi Balaka

Dalam tugas ini, Anda akan mempelajari *text preprocessing* yang umum dilakukan dalam pengembangan sistem *Information Retrieval*. Terdapat tiga proses yang akan dibahas, yakni tokenisasi, *stemming*, dan *stop word removal*.

*Notebook* ini terdiri atas dua bagian: contoh kode dan soal. Bagian pertama akan menjadi landasan untuk Anda mengerjakan soal yang ada. Selamat mengerjakan dan semoga bermanfaat!

## Bagian 1: Contoh Kode

Bagian ini akan membahas mengenai contoh pemrosesan teks. Silakan pahami dan lengkapi beberapa fungsi untuk mengerjakan soal-soal di Bagian 2.

In [ ]:
# Install package jika belum ada dan pastikan Anda menggunakan Python >= 3.7
# !pip install pandas==2.2.2
# !pip install PySastrawi==1.2.0

In [ ]:
# Import packages yang diperlukan
import re
import pandas as pd

from collections import defaultdict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.Stemmer.CachedStemmer import CachedStemmer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

### 1.1: Tokenisasi

Tokenisasi adalah proses mengubah teks menjadi serangkaian token. Ada berbagai perspektif terkait apa yang dimaksud dengan token. Salah satu makna yang umum digunakan adalah setiap token merepresentasikan suatu kata tertentu. Namun, bisa juga token merepresentasikan informasi yang lebih granular seperti karakter.

Pada bagian ini, kita akan berfokus pada dua metode saja, yakni tokenisasi dengan *regular expression* (regex) dan Byte-Pair Encoding (BPE).

#### Metode 1: Tokenisasi dengan regex

In [ ]:
def tokenize_text_regex(text: str):
    """
    Tokenisasi teks berdasarkan pola tokenizer_pattern
    """
    tokenizer_pattern = r"TODO: pola"  # Memecah teks berdasarkan sekuens alfanumerik
    # TODO: implementasi


# Contoh tokenisasi pada suatu teks
text_1 = "Saya sedang mengerjakan tugas pada mata kuliah Temu-Balik Informasi."
tokens_by_regex_tokenizer = tokenize_text_regex(text_1)
print(f"Text: {text_1}")
print(f"Tokens of the text: {tokens_by_regex_tokenizer}")

#### Metode 2: Tokenisasi BPE

Referensi:
- [Text Processing by Jurafsky](https://web.stanford.edu/~jurafsky/slp3/slides/2_TextProc_2023.pdf)
- [BPE Tutorial by HuggingFace](https://huggingface.co/learn/nlp-course/en/chapter6/5)

In [ ]:
def get_word_freqs(corpus: list[str]):
    """
    Menghasilkan frekuensi dari tiap kata dengan format:
    {"kata1": f_1, "kata2": f_2, ...}
    """
    # TODO: implementasi


# Contoh pemanggilan
corpus = [
    "low low low low low lowest lowest newer newer newer",
    "newer newer newer wider wider wider new new",
]
word_freqs = get_word_freqs(corpus)
print(f"Contoh word_freqs: {word_freqs}")

In [ ]:
def get_initial_vocabulary(word_freqs: dict[str, int]):
    """
    Membuat vocabulary awal yang berisi karakter dari word_freqs
    """
    vocabulary = set()
    for word in word_freqs.keys():
        for letter in word:
            vocabulary.add(letter)
    vocabulary = sorted(vocabulary)
    return vocabulary


# Contoh pemanggilan
init_vocab = get_initial_vocabulary(word_freqs)
print(f"Contoh initial vocab: {init_vocab}")

In [ ]:
def compute_pair_freqs(word_splits: dict[str, list[str]], word_freqs: dict[str, int]):
    """
    Menghitung frekuensi tiap pair karakter
    """
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = word_splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs


# Contoh pemanggilan
word_splits: dict[str, list[str]] = {
    word: [c for c in word] for word in word_freqs.keys()
}
print(f"Word splits: {word_splits}")
pair_freqs = compute_pair_freqs(word_splits, word_freqs)
print(f"Contoh pair_freqs: {pair_freqs}")

In [ ]:
def merge_split(a: str, b: str, split: list[str]):
    """
    Melakukan merging a dan b pada split
    """
    # TODO: implementasi

In [ ]:
def merge_pair(a: str, b: str, word_splits: dict[str, list[str]]):
    """
    Merging a dan b pada word_splits
    """
    for word in word_splits.keys():
        split = word_splits[word]
        if len(split) > 1:
            word_splits[word] = merge_split(a, b, split)
    return word_splits

word_splits_new = merge_pair("h", "a", word_splits.copy())
print(f"Contoh word_splits sebelum merging: {word_splits}")
print(f"Contoh word_splits setelah merging: {word_splits_new}")

In [ ]:
def train_bpe(corpus: list[str], num_of_merges: int):
    """
    Melatih tokenizer BPE pada korpus untuk mendapatkan aturan merging
    yang akan digunakan untuk melakukan tokenisasi nantinya
    """
    merge_rules: list[tuple[str, str]] = []
    word_freqs: dict[str, int] = get_word_freqs(corpus)
    vocab: list[str] = get_initial_vocabulary(word_freqs)
    word_splits = {word: [c for c in word] for word in word_freqs.keys()}

    for i in range(num_of_merges):
        try:
            pair_freqs = compute_pair_freqs(word_splits, word_freqs)
            best_pair = ""
            max_freq = None

            for pair, freq in pair_freqs.items():
                if max_freq is None or max_freq < freq:
                    best_pair = pair
                    max_freq = freq

            merge_rules.append(best_pair)
            vocab.append("".join(best_pair))
            word_splits = merge_pair(
                best_pair[0], best_pair[1], word_splits
            )
        except:
            print(f"Iteration stops early at {i}")
            break
    return vocab, merge_rules

In [ ]:
# Coba kita latih dengan teks pada corpus contoh
vocab, merge_rules = train_bpe(corpus, 10)

In [ ]:
def tokenize_bpe(vocab: list[str], merge_rules: list[tuple[str, str]], text: str):
    """
    Melakukan tokenisasi pada corpus berdasarkan vocab dan merge_rules yang
    didapatkan dari proses training
    """
    tokenized_text = []
    for word in get_word_freqs([text]).keys():
        word_split: list[str] = []
        for char in word:
            if char in vocab:
                word_split.append(char)
            else:
                word_split.append("<UNK>")  # Karakter yang tidak dikenali

        for merge_rule in merge_rules:
            merge_str = "".join(merge_rule)
            i = 0
            while i < len(word_split) - 1:
                if (
                    word_split[i] == merge_rule[0]
                    and word_split[i + 1] == merge_rule[1]
                ):
                    word_split = word_split[:i] + [merge_str] + word_split[i + 2 :]
                else:
                    i += 1
        tokenized_text += word_split
    return tokenized_text

In [ ]:
# Kita tes tokenisasi suatu teks
text_2 = "lower newer"
tokens_by_bpe_tokenizer = tokenize_bpe(vocab, merge_rules, text_2)
print(tokens_by_bpe_tokenizer)

### 1.2: *Stemming*

*Stemming* merupakan proses transformasi kata dari bentuk infleksi ke bentuk dasar.
Secara umum, cara kerjanya adalah dengan "memotong" tambahan seperti afiks berdasarkan
aturan tertentu.

Contoh dalam Bahasa Indonesia: ["memakan", "dimakan", "termakan"] -> "makan"

Untuk task ini, kita akan menggunakan [PySastrawi](https://github.com/har07/PySastrawi).

In [ ]:
# Mendefinisikan stemmer
factory: StemmerFactory = StemmerFactory()
stemmer: CachedStemmer = factory.create_stemmer()

In [ ]:
def stem_tokens(stemmer: CachedStemmer, tokens: list[str]):
    """
    Melakukan stemming pada tokens
    """
    stemmed_tokens: list[str] = [
        stemmer.stem(token) if token else "" for token in tokens
    ]
    stemmed_tokens_without_empty_string: list[str] = [
        token for token in stemmed_tokens if not ((token == "") or (token == None))
    ]
    return stemmed_tokens_without_empty_string

In [ ]:
# Contoh pemanggilan
stemmed_tokens = stem_tokens(stemmer, tokens_by_regex_tokenizer)
print(f"Kumpulan token sebelum stemming: {tokens_by_regex_tokenizer}")
print(f"Kumpulan token setelah stemming: {stemmed_tokens}")

### 1.3: *Stop Words Removal*

*Stop word* merupakan kata-kata yang umumnya memiliki frekuensi yang sangat tinggi dalam teks namun tidak memberikan informasi yang signifikan. Oleh karena itu, kata-kata tersebut seringkali dihapus
agar menyisakan informasi yang "penting" saja.

Contoh *stop words* pada Bahasa Indonesia: "yang", "di", "pada"

Kita akan menggunakan PySastrawi juga untuk *task* ini.

In [ ]:
# Mendefinisikan kumpulan stop words
stop_factory = StopWordRemoverFactory()
stop_words = set(stop_factory.get_stop_words())

In [ ]:
def remove_stop_words(tokens: list[str], stop_words: set[str]):
    """
    Menghapus stop words dari kumpulan token
    """
    tokens_without_stop_words = [token for token in tokens if token not in stop_words]
    return tokens_without_stop_words

In [ ]:
# Contoh pemanggilan
stop_words_removed_tokens = remove_stop_words(stemmed_tokens, stop_words)
print(f"Kumpulan token sebelum stop words dihapus: {stemmed_tokens}")
print(f"Kumpulan token setelah stop words dihapus: {stop_words_removed_tokens}")

### 1.4: Kombinasi 1.1 - 1.3

Semua fungsi di atas bisa dirangkai menjadi suatu *pipeline*.

In [ ]:
# Contoh melakukan pemrosesan menggunakan regex tokenizer; Anda bisa gunakan BPE sebagai alternatif
def text_processing_pipeline(
    text: str, pattern: str, stemmer: CachedStemmer, stop_words: set[str]
):
    tokens = tokenize_text_regex(text, pattern)
    tokens = stem_tokens(stemmer, tokens)
    tokens = remove_stop_words(tokens, stop_words)
    return tokens

In [ ]:
# Contoh menjalankan pipeline
tokens_by_pipeline = text_processing_pipeline(
    text_1, tokenizer_pattern, stemmer, stop_words
)
print(f"Teks: {text_1}")
print(f"Tokens: {tokens_by_pipeline}")

Hasil akhir dari penerapan tersebut adalah data (tokens) yang siap digunakan untuk pemrosesan selanjutnya dalam aplikasi IR.

## Bagian 2: Soal

Pada bagian ini, Anda diminta untuk menerapkan apa yang sudah Anda pelajari pada
Bagian 1. Dalam menjawab pertanyaan, Anda dibebaskan untuk menambah *cell* baru sesuai kebutuhan.
Selain itu, Anda juga dibebaskan untuk menambahkan *package* lain jika dibutuhkan
(pastikan untuk menuliskan *package* yang digunakan dan versinya).

In [ ]:
# Install package lain jika perlu

In [ ]:
# Load dataset yang diadaptasi dari https://huggingface.co/datasets/roneneldan/TinyStories
df = pd.read_csv("stories.csv")

# Gunakan korpus berikut untuk pengerjaan soal
corpus = df["story"]

### Soal 1: Menyesuaikan Komponen *Preprocessing* untuk Bahasa Inggris

Perhatikan bahwa `corpus` yang digunakan berbahasa Inggris. Oleh karena itu, beberapa
komponen dari Bagian 1.1 - 1.3 perlu disesuaikan. Silakan Anda pertimbangkan apa
saja yang perlu diubah dan apa yang bisa digunakan untuk Bahasa Inggris juga.
Implementasikan kodenya.

In [ ]:
# TODO: kode

### Soal 2: Membandingkan *Pipeline* Berbasiskan Regex dan BPE

Dengan komponen yang sudah Anda sesuaikan pada Soal 1, implementasikan dua *pipeline*
dengan *tokenizer* berbeda (antara regex atau BPE). Untuk BPE, latih pada `corpus`
dan gunakan suatu nilai `num_of_merges`. Jelaskan alasan Anda menggunakan nilai
`num_of_merges` tersebut. Lalu, jalankan kedua *pipeline* pada tiga teks yang dipilih
secara acak dari `corpus`. Analisis kelebihan dan kekurangan dari masing-masing
*pipeline* pada ketiga teks tersebut.

In [ ]:
# TODO: kode

TODO: Analisis

### Soal 3: Membangun *Stop Words*

Dengan memanfaatkan *tokenizer* regex, Anda diminta untuk mengumpulkan seluruh
token unik beserta frekuensi kemunculannya dari `corpus`. Berdasarkan frekuensi
kemunculan ini, ambil *top-200* token yang memiliki frekuensi tertinggi sebagai
*stop words*.

Tampilkan *stop words* Anda dan bandingkan dengan *stop words* yang Anda gunakan
di Soal 1 dan 2. Jelaskan hasil observasi Anda.

In [ ]:
# TODO: kode

TODO: observasi